## Import Statements

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

## Get Data

In [12]:
c1 = open('S6_feats-S6-C1-feats_5scut.pkl', 'rb')
c2 = open('S6_feats-S6-C2-feats_5scut.pkl', 'rb')
c3 = open('S6_feats-S6-C3-feats_5scut.pkl', 'rb')
c4 = open('S6_feats-S6-C4-feats_5scut.pkl', 'rb')

d1 = pd.read_pickle(c1)
d2 = pd.read_pickle(c2)
d3 = pd.read_pickle(c3)
d4 = pd.read_pickle(c4)

data = pd.concat([d1, d2, d3, d4])

## Clean Data

In [16]:
junk_tics = [91924940, 234661768, 281897530, 153545882, 282051280, 153545868, 91863758, 50589230, 287769916, 31742413,
             322895826, 119690108, 322895794, 119689919, 302761551, 281980550, 91922911, 281979571, 287957207, 281980466,
             259543633, 31816351, 281980360, 47346711, 397186531, 119555751, 119689376, 119690511, 120381685, 134199361,
             142105922, 142105969, 143143921, 146911567, 167303776, 167338401, 167362377, 176980088, 177022355, 219184692,
             269205934, 269208586, 269273824, 269274274, 279319817, 280867759, 287957147, 322742255, 322897564, 427394477]

var_tics = [387225893, 11295270, 149250423, 381975513, 100479926, 281853128, 370039101, 31473897, 271553990, 238230361,
            153096567, 443418729, 138979820, 287532827, 234037006, 119364133, 149349182, 30928630, 271973253, 386320787,
            365642947, 200518399, 317091666, 32929951, 442933194, 200515351, 438521908, 332919383, 427349922, 388935228,
            50529755, 284283865, 200592903, 11286019, 276664304, 4352884, 50788340, 279578258, 200516548, 4357441,
            427348371, 199898021, 200478060, 72921546, 762225404, 457250777, 92017453, 382144165, 40405809, 158733064]

eb_tics = [263272042, 251238463, 141213462, 53328873, 67038755, 410901165, 283005536, 147260698, 235457825, 144542315,
           151354333, 293776750, 247870021, 220025156, 234879341, 31403133, 143177176, 229306812, 80608317, 281803169,
           24623945, 220241814, 52579544, 67192771, 234625764, 100509705, 100682066, 100682887, 102282850, 102405788,
           102411949, 102541006, 102571243, 11293508, 11412481, 11693134, 117666617, 117765901, 117883206, 11807985,
           11808371, 118929099, 119461300, 119549533, 12036808, 281804470, 282083645, 282263788, 282381271, 284999958]

dip_tics = [72828661, 11286045, 388935312, 72833961, 4357786, 11286071, 427395986, 11286063, 427377818, 158822738,
            52209961, 436013289, 50530445, 459989537, 279640885, 388935622, 220322494, 206901502, 427377817, 158764834,
            436159171, 302672869, 264541453, 309795677, 207045924, 159089190, 34397579, 457231768, 4206238, 349924407,
            200639492, 317873721, 436248202, 200591757, 284158141, 302762301, 427400158, 427395634, 365643370, 427380721,
            427334205, 200640498, 207017053, 24775707, 11197417, 264739659, 457251847, 457250759, 302830280, 144666684]

In [17]:
data['label'] = None
data.loc[junk_tics, 'label'] = 'JUNK'  # Junk
data.loc[var_tics, 'label'] = 'VAR'  # Variable
data.loc[eb_tics, 'label'] = 'EB'  # Eclipsing Binary
data.loc[dip_tics, 'label'] = 'DIP'  # Dipper

In [18]:
data_labeled = data.dropna(subset=['label'])

In [19]:
labels = np.array(data_labeled['label'])
data_unlabeled = data_labeled.drop('label', axis = 1)
feature_list = list(data_unlabeled.columns)

## Random Forest

In [20]:
train_features, test_features, train_labels, test_labels = train_test_split(data_unlabeled, labels, test_size = 0.5, random_state = 0)

In [24]:
rf = RandomForestClassifier(n_estimators = 5, random_state=0)
rf.fit(train_features, train_labels)
pred_labels = rf.predict(test_features)

print("MODEL ACCURACY: ", metrics.accuracy_score(test_labels, pred_labels))

MODEL ACCURACY:  0.86


## Get Dipper Candidates

In [25]:
data_all = data.drop('label', axis=1)
pred = rf.predict(data_all)

In [26]:
unique, counts = np.unique(pred, return_counts=True)
for i in range(0, len(unique)):
    print(unique[i], ':\t', counts[i])

DIP :	 41106
EB :	 69172
JUNK :	 2616970
VAR :	 52477


In [27]:
data_pred = data_all.copy()
data_pred['prediction'] = pred

In [28]:
confidence = rf.predict_proba(data_all)
data_pred['dip_confidence'] = confidence[:, 0]

In [29]:
data_dip = data_pred[data_pred['prediction'] == 'DIP']
data_dip = data_dip.sort_values('dip_confidence', ascending=False)
data_dip

,longtermtrend,meanmedrat,skews,varss,coeffvar,stds,numoutliers,numnegoutliers,numposoutliers,numout1s,...,sautocorrcoef,autocorrcoef,flatmean,tflatmean,roundmean,troundmean,roundrat,flatrat,prediction,dip_confidence
427377818,129.721323,1.345634,0.987055,0.222756,-0.106987,0.776875,0.957712,-0.894645,3.246358,19.0,...,-1.357347,0.794371,0.219082,0.811178,-2.676161,1.525118,2.444883,0.047997,DIP,1.0
220322339,13.383646,0.094584,0.973569,0.021303,-0.000062,0.066902,0.908787,-0.999427,0.621773,16.0,...,-0.627287,0.944708,0.021220,0.073616,-0.003994,0.026805,0.038515,0.000450,DIP,1.0
237602818,6.281984,0.022612,0.980632,0.007162,-0.000751,0.035932,0.854231,-0.984867,-1.182578,26.0,...,0.029283,1.011927,0.007140,0.042064,0.536771,0.001226,0.003651,0.000051,DIP,1.0
144728975,7.613080,0.047698,0.984597,0.013209,-0.007919,0.050833,0.850498,-0.891116,-1.121463,16.0,...,0.199565,1.000932,0.013173,0.059768,-0.368349,0.004567,0.009242,0.000174,DIP,1.0
234089891,4.310072,0.038779,0.982898,0.008161,-0.001788,0.080365,0.959617,-0.962660,0.043017,28.0,...,0.221899,1.099819,0.008154,0.083747,0.778314,0.001395,0.003599,0.000066,DIP,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50792974,15.331776,0.059187,0.974126,0.012304,0.004785,0.050832,1.089542,-1.049970,1.706593,21.0,...,-0.054935,0.995917,0.012271,0.046655,-1.059675,0.006578,0.016199,0.000151,DIP,0.4
242066344,37.389562,0.072375,0.806722,0.020132,0.013989,0.170037,0.207746,-1.040636,9.957060,35.0,...,0.323273,1.037074,0.020012,0.818483,1.109531,0.136123,0.259437,0.000400,DIP,0.4
242065936,5.084903,0.018521,0.910945,0.002855,0.002436,0.015935,1.348477,-1.077781,27.373406,28.0,...,0.077544,1.050429,0.002854,0.011817,-0.333342,0.000702,0.001759,0.000008,DIP,0.4
167970588,19.430053,0.018781,0.463350,0.005079,0.003117,0.041856,0.893686,-1.016515,-1.071857,40.0,...,0.041088,1.015127,0.005076,0.046836,-1.114021,0.013392,0.049586,0.000026,DIP,0.4


In [30]:
data_dip.to_csv('rf_dip_candidates.csv', columns=['dip_confidence'])